# MIE1613 Project: Developing the FIG-toronto simulation

In [1]:
# Import Libraries
# Data
import numpy as np
import scipy as sp
from scipy import stats
import pandas as pd
import random

import re
from copy import deepcopy
#from tqdm import tqdm
import tqdm.notebook as tq

#Viz
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
sns.set(font='Helvetica') # Futura? Calibri Light? 
sns.set_style("white")
sns.set_theme(style='ticks')
sns.set_context('talk')

c:\Users\Keagan Rankin\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
c:\Users\Keagan Rankin\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Keagan Rankin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


## Structuring functions
Want to create simulation using the principles and framework from the class. I will create functions that do the input fitting, simulation action, and output operations, and then store things as objects

### Input functions

These functions import the required data

In [2]:
def import_emission_data(factor_range='likely', year=2023):
    """
    imports and formats the emboded and operational 
    emissions per unit data for each building type.
    factor_range = (min, likely, max) option for the embodied
    ghg emission factors of materials. Used for sensitivity analysis.
    year = year for embodied emissions data.
    """
    # Initializations
    op_path = 'data/CALCULATED_operational_intensity_2020_2050.csv'
    emb_path = 'data/embodied_'+factor_range+'.csv'

    # Embodied Emissions - load
    embodied = pd.read_csv(emb_path, index_col=0)

    # Operational Emissions - load pre-calculated table 2020-2050
    operational = pd.read_csv(op_path, index_col=0).loc[2023:]

    return (embodied, operational)


In [3]:
emit_e, emit_o = import_emission_data(factor_range='likely', year=2023)

In [4]:
def percent_reduction_operational(operational_df):
    """
    return a table of the percent reduction
    of operational emissions each year compared 
    to the base year. Used when chaining time periods.
    """
    return operational_df.div(operational_df.iloc[0], axis=1)

percent_reduction_operational(emit_o).head()

,detached_gju,attached_gju,apartment_gju
2023,1.000000,1.000000,1.000000
2024,0.923240,0.909001,0.861270
2025,0.880167,0.867796,0.815109
2026,0.825500,0.803609,0.755476
2027,0.772036,0.740944,0.696773


This fits emission inputs using MLE

In [5]:
def embodied_fitter_mle(data, distribution=stats.exponnorm):
    """
    Fit data to distribution
    using maxmimum likelyhood estimation.
    distribution = scipy type dist to fit.
    data = imported embodied emissions data from import_emission_data().

    fits the three major housing types, returns parameters.
    """
    # three housing types
    lmh = ['Single Family','Missing Middle','Mid High Rise']

    # loop and fit each
    parameters = {}
    for house_type in lmh:
        d_col = 'ghg_per_unit'

        data_i = data.loc[data['labels_general']==house_type][d_col]
        parameters[house_type] = distribution.fit(data_i)
    
    return parameters


In [6]:
embodied_params_exp_norm = embodied_fitter_mle(data=emit_e.loc[emit_e['year']==2023])
embodied_params_exp_norm

{'Single Family': (1.8027800847884894, 43232.896492383894, 12680.214166091042),
 'Missing Middle': (1.9692317430620259, 14671.068157312147, 5765.404652034498),
 'Mid High Rise': (1.774038706436527, 20220.33723155928, 4305.313479593345)}

time series data for housing starts

In [7]:
# def build_houses

## Simulation functions
these functions run the simulation logic

In [8]:
class Simulation:
    
    def __init__(self):
        # random seed
        self.seed = 1614

        # year for single period simulation
        self.year = 2023

        # distribution for embodied emissions
        self.e_dist = stats.exponnorm

        # lmh name and percentages
        self.lmh = ['Single Family','Missing Middle','Mid High Rise']
        self.nlmh = [0.09, 0.24, 0.67]

        # input data - embodied and operational
        self.e_data = 0
        self.o_data = 0
        

building block function: getting the operational and embodied emissions for a certain number of built houses

In [9]:
def build_embodied(n, e_params, e_dist=stats.exponnorm):
    """
    Building block function that calculates embodied
    emissions from n constructed houses in time period T.
    n = # units to build
    e_dist = distribution of embodied emissions (kgCO2e/unit)
    e_params = embodied distribution parameters
    """
    # sample embodied
    e_samples = e_dist.rvs(*e_params, size=n)
    e_emissions = np.sum(e_samples) # sum of CO2e

    return e_emissions


def build_operational(n, o_factor):
    """
    Building block function that calculates operational (energy)
    emissions from n constructed houses in time period T
    n = # of units
    o_factor = operational CO2e per unit in T
    """
    # return performance variables
    o_emissions = o_factor * n # CO2e/unit * unit/time

    return o_emissions

In [10]:
def build_houses(n, o_factor, e_params, e_dist=stats.exponnorm):
    """
    Building block function
    Sample houses equal to the number that
    must be built in time period T. Return performance
    measure theta = E_embodied + E_operational
    n = # of units to build.
    e_dist = embodied distribution.
    e_params = embodied distribution parameters.
    o_factor = operational CO2e per unit per T.
    """
    # fix random
    #np.random.seed(seed=1613)

    # sample embodied
    e_samples = e_dist.rvs(*e_params, size=n)

    # return performance variables
    o_emissions = o_factor * n # CO2e/unit * unit/time
    e_emissions = np.sum(e_samples) # sum of CO2e

    # in kgCO2e
    return {'embodied':e_emissions, 'operational':o_emissions}


In [11]:
build_houses(n=1, o_factor=emit_o.loc[2023,'detached_gju']*10**3, e_params=embodied_params_exp_norm['Single Family'])

{'embodied': 76310.75267732216, 'operational': 20318.918160859637}

Extend this function to allow for control over the percentage of L,M,H housing type mix built in the time period.

In [12]:
def get_operational_factor(operational_df, year):
    """
    Function returns the lmh
    CO2e/unit factor from imported operational
    emissions data for easy input into functions.
    operational_df = import_emission_data() output [1]
    year = year int
    """
    o_dict = dict(zip(['Single Family','Missing Middle','Mid High Rise'], 
                        operational_df.loc[year,['detached_gju','attached_gju','apartment_gju']].values*10**3))
    
    return o_dict
    

In [13]:
get_operational_factor(emit_o, 2023)

{'Single Family': 20318.918160859637,
 'Missing Middle': 2457.3706582948953,
 'Mid High Rise': 1125.0579625281252}

In [14]:
def split_n_lmh(n, l=0.09, m=0.24, h=0.67):
    """
    splits # built n into the three proportions of
    l (single family), m (low-rise multi-unit), 
    h (mid high rise). Rounding up.
    """
    lmh = ['Single Family','Missing Middle','Mid High Rise']
    # get n for each type
    n_l = int(np.ceil(n*l))
    n_m = int(np.ceil(n*m))
    n_h = int(np.ceil(n*h))
    n_dict = dict(zip(lmh,[n_l,n_m,n_h]))

    return n_dict

Functions gets operational and embodied seperately for lmh

In [15]:
def build_embodied_lmh(n, e_params_dict, e_dist=stats.exponnorm, l=0.09, m=0.24, h=0.67):
    """
    Extend build_embodied to allow for control of
    percentage of housing types built 
    (single family, low-rise multi-unit, mid/high rise)

    dict form -> {'Single Family','Missing Middle','Mid High Rise'}
    e_params_dict = full dict from embodied_params_exp_norm
    """
    lmh = ['Single Family','Missing Middle','Mid High Rise']

    # split n
    n_dict = split_n_lmh(n, l, m, h)

    lmh_embodied = {}
    for h_type in lmh:
        b = build_embodied(n=n_dict[h_type], 
                           e_params=e_params_dict[h_type],
                           e_dist=e_dist)
        # add n_dict to the output
        #lmh_embodied['built'] = n_dict[h_type]
        lmh_embodied[h_type] = b

    return lmh_embodied


def build_operational_lmh(n, o_factor_dict, l=0.09, m=0.24, h=0.67):
    """
    Extend build_operational to allow for control of
    percentage of housing types built 
    (single family, low-rise multi-unit, mid/high rise)

    dict form -> {'Single Family','Missing Middle','Mid High Rise'}
    o_factor_dict = dictionary of operational factors
    """
    lmh = ['Single Family','Missing Middle','Mid High Rise']

    # split n
    n_dict = split_n_lmh(n, l, m, h)

    lmh_operational = {}
    for h_type in lmh:
        b = build_operational(n=n_dict[h_type],
                              o_factor=o_factor_dict[h_type])
        # add n_dict to the output
        lmh_operational[h_type] = b

    return lmh_operational

In [16]:
build_embodied_lmh(n=10, e_params_dict=embodied_params_exp_norm)

{'Single Family': 115897.46393369412,
 'Missing Middle': 108233.95293118658,
 'Mid High Rise': 169007.55972245854}

In [17]:
build_operational_lmh(n=10, o_factor_dict=get_operational_factor(emit_o, 2023))

{'Single Family': 20318.918160859637,
 'Missing Middle': 7372.111974884686,
 'Mid High Rise': 7875.405737696877}

This function combines operational and embodied

In [18]:
def build_houses_lmh(n, o_factor_dict, e_params_dict, e_dist=stats.exponnorm, l=0.09, m=0.24, h=0.67):
    """
    Extend build_houses to allow for control of
    percentage of housing types built 
    (single family, low-rise multi-unit, mid/high rise)

    dict form -> {'Single Family','Missing Middle','Mid High Rise'}
    o_factor_dict = dictionary of operational factors 
    e_params_dict = full dict from embodied_params_exp_norm
    """
    lmh = ['Single Family','Missing Middle','Mid High Rise']
    #np.random.seed(seed=1614)

    # get n for each type
    n_dict = split_n_lmh(n, l, m ,h)
    
    # build each type and return emissions, store in nested type dict
    lmh_emissions = {}
    for h_type in lmh:
        b = build_houses(n=n_dict[h_type], 
                         o_factor=o_factor_dict[h_type], 
                         e_params=e_params_dict[h_type],
                         e_dist=e_dist)
        # add n_dict to the output
        b['built'] = n_dict[h_type]
        lmh_emissions[h_type] = b

    return lmh_emissions


In [19]:
foo = build_houses_lmh(n=10, 
                       o_factor_dict=get_operational_factor(emit_o, 2023), 
                       e_params_dict=embodied_params_exp_norm)

foo

{'Single Family': {'embodied': 65359.6360470026,
  'operational': 20318.918160859637,
  'built': 1},
 'Missing Middle': {'embodied': 49636.81292742955,
  'operational': 7372.111974884686,
  'built': 3},
 'Mid High Rise': {'embodied': 202327.1823406137,
  'operational': 7875.405737696877,
  'built': 7}}

### Yearly CHAINING 
together years requires carry over of operational multiplied by the reduction factor from the base year. It also requires clean imports every year.
<br>
- For operational emissions, we just need the base year factors (2023). These can be multiplied by the percent_reduction_operational() frame each year to reduce the emissions
- For embodied, we just calculate and store them every year

In [20]:
def chain_periods_build_lmh(y, b, e_data, o_data, verbose=False):
    """
    chain together multiple time periods of the simulation
    (e.g. years), store cumulative emissions, calculate 
    the continued operational output with the delta reduction
    each year.
    y = vector of time periods.
    b = vector of # of houses built in time period.
    e_data = embodied emissions data (labelled by year)
    o_data = operational emissions data (labelled by year)
    crn = common random numbers (default false)
    verbose = extra text outputs
    """
    # setup for storing variables
    raw = []
    embodied = []
    operational = [] # energy from houses built in current AND previous years.
    built = []

    # counting built
    nl_cum = 0
    nm_cum = 0
    nh_cum = 0

    # for each year
    for en, i in enumerate(y):
        # FIT INPUT DATA
        # MLE for embodied, factor for operational
        embodied_params_y = embodied_fitter_mle(data=e_data.loc[e_data['year']==i])
        operational_factor = get_operational_factor(o_data, i)

        # get the number of starts for each housing type
        n_lmh_i = split_n_lmh(b[en], l=0.09, m=0.24, h=0.67)


        # BUILD HOUSES, CALC PERFORMANCE VARIABLES
        # embodied emissions for the given year
        embodied_i = build_embodied_lmh(n=b[en], e_params_dict=embodied_params_y)

        # operational emissions of cumulative houses built ##########
        # energy emissions from houses built this year.
        operational_i = build_operational_lmh(n=b[en], o_factor_dict=operational_factor)
        # energy emissions from houses built in previous years.
        operational_cum_l =  build_operational(nl_cum, operational_factor['Single Family'])
        operational_cum_m =  build_operational(nm_cum, operational_factor['Missing Middle'])
        operational_cum_h =  build_operational(nh_cum, operational_factor['Mid High Rise'])

        if verbose == True:
            print('year', i)
            print('nlcum', nl_cum)
            print('opi', operational_i)
            print('opm', operational_cum_m)


        # STORE OUTPUTS
        # full data
        raw.append({'year':i,
                    'embodied':embodied_i, 
                    'operational_y':operational_i,
                    'operational_cum':operational_i['Single Family']+operational_i['Missing Middle']+operational_i['Mid High Rise']+
                           operational_cum_l+operational_cum_m+operational_cum_h,
                    'num_built':n_lmh_i})

        # emissions each year
        embodied.append(embodied_i['Single Family']+embodied_i['Missing Middle']+embodied_i['Mid High Rise'])
        operational.append(operational_i['Single Family']+operational_i['Missing Middle']+operational_i['Mid High Rise']+
                           operational_cum_l+operational_cum_m+operational_cum_h)

        # number built
        built.append(n_lmh_i['Single Family'] + n_lmh_i['Missing Middle'] + n_lmh_i['Mid High Rise'])
        # broken down by type for energy calculation
        nl_cum += n_lmh_i['Single Family']
        nm_cum += n_lmh_i['Missing Middle']
        nh_cum += n_lmh_i['Mid High Rise']

        
        print(operational)

    
    return (raw, embodied, operational, built)


In [21]:
output = chain_periods_build_lmh([2023,2024], [100,1000], emit_e, emit_o, verbose=True)

year 2023
nlcum 0
opi {'Single Family': 182870.26344773674, 'Missing Middle': 58976.89579907749, 'Mid High Rise': 75378.88348938439}
opm 0.0
[317226.04273619864]
year 2024
nlcum 9
opi {'Single Family': 1688331.0875288984, 'Missing Middle': 536100.5090577394, 'Mid High Rise': 649216.0201728678}
opm 53610.05090577394
[317226.04273619864, 3161012.378435456]


In [22]:
#pd.concat([pd.DataFrame(r) for r in raw], axis=0)
#np.sum(output[3])
output[2]

[317226.04273619864, 3161012.378435456]

In [23]:
np.sum([o['num_built']['Single Family'] for o in output[0]])

99

## Final
The above code is enough to run the basic sim. I transport this to a object structure to make it easier to run and to reduce input complexity. The class(es) are under packages/project_packages. This file can now be used for simulation experiments

In [24]:
from packages.project_package.mie_importer import *
from packages.project_package.mie_simulation import *

In [25]:
proj_importer = MIEImporter()
proj_simulator = MIESimulation()

Importer created.
Sim object created. Initialize self.e_data and self.o_data


In [26]:
data_embodied, data_operational = proj_importer.import_emission_data()

In [27]:
proj_simulator.e_data = data_embodied
proj_simulator.o_data = data_operational

foo2 = proj_simulator.chain_periods_build_lmh([2023,2024], [100,1000])

In [28]:
foo2['E_o']

[317226.04273619864, 3161012.378435456]